# Лабораторная работа №5
## Выполнил студент группы БФИ2301 Семин Артем Павлович


### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [15]:
import heapq
from copy import deepcopy

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [16]:
text = input("Введите текст: ")
pattern = input("Введите паттерн: ")
sensitive = input("Чувствителен к регистру? (y/n): ").lower()

In [17]:
def kmp_search(text, pattern, case_sensitive=True):
    if not case_sensitive:
        text = text.lower()
        pattern = pattern.lower()
    
    n = len(text)
    m = len(pattern)
    
    if m == 0:
        return list(range(n + 1))
    
    # Построение префикс-функции
    lps = [0] * m
    j = 0
    for i in range(1, m):
        while j > 0 and pattern[i] != pattern[j]:
            j = lps[j - 1]
        if pattern[i] == pattern[j]:
            j += 1
        lps[i] = j
        
    # Поиск подстроки
    result = []
    j = 0
    for i in range(n):
        while j > 0 and text[i] != pattern[j]:
            j = lps[j - 1]
        if text[i] == pattern[j]:
            j += 1
        if j == m:
            result.append(i - m + 1)
            j = lps[j - 1]
    return result

In [18]:
kmp_search(text, pattern, sensitive == 'y')
kmp_result = kmp_search(text, pattern, sensitive == 'y')
print("KMP-search positions:", kmp_result)

KMP-search positions: [0, 4, 7, 10]


#### Упрощенный алгоритм Бойера-Мура

In [19]:
def build_shift_table(pattern):
    """Строит таблицу смещений для алгоритма Бойера-Мура."""
    table = {}
    length = len(pattern)
    
    # Для каждого символа в шаблоне определяем его самое правое вхождение
    for i, char in enumerate(pattern[:-1]):
        table[char] = length - i - 1
    
    # Если символа нет в таблице, смещение на длину шаблона
    table.setdefault(pattern[-1], length)
    return table

In [20]:
def boyer_moore_search_all(text, pattern, case_sensitive=True):
    if not case_sensitive:
        text = text.lower()
        pattern = pattern.lower()
    
    if not pattern:
        return [0]  # Пустая подстрока найдена в начале строки
    
    shift_table = build_shift_table(pattern)
    pattern_len = len(pattern)
    text_len = len(text)
    i = 0  # Позиция в тексте
    occurrences = []
    
    while i <= text_len - pattern_len:
        j = pattern_len - 1  # Начинаем сравнение с конца шаблона
        
        # Сравниваем символы с конца шаблона
        while j >= 0 and text[i + j] == pattern[j]:
            j -= 1
        
        if j < 0:
            # Найдено совпадение - добавляем позицию
            occurrences.append(i)
            # Сдвигаем на 1 для поиска следующего вхождения
            i += 1
        else:
            # Сдвигаем индекс i согласно таблице смещений
            shift = shift_table.get(text[i + j], pattern_len)
            i += max(1, shift - (pattern_len - 1 - j))
    
    return occurrences if occurrences else [-1]  # Возвращаем список или [-1] если не найдено

In [21]:
print(boyer_moore_search_all(text, pattern, sensitive == 'y'))

[0, 4, 7, 10]


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [22]:
def is_solvable(puzzle):
    inversions = 0
    n = len(puzzle)
    zero_pos = puzzle.index(0)
    zero_row_from_bottom = 4 - (zero_pos // 4)  # Считаем строку с НИЗА (1-4)

    for i in range(n):
        if puzzle[i] == 0:
            continue
        for j in range(i + 1, n):
            if puzzle[j] != 0 and puzzle[i] > puzzle[j]:
                inversions += 1

    # Правило: если пустая клетка в нечётной строке снизу, инверсий должно быть чётно
    return (zero_row_from_bottom % 2 == 1) == (inversions % 2 == 0)

In [23]:
def manhattan_distance(puzzle):
    distance = 0
    for i in range(16):
        if puzzle[i] == 0:
            continue
        x, y = i % 4, i // 4
        target_x = (puzzle[i] - 1) % 4
        target_y = (puzzle[i] - 1) // 4
        distance += abs(x - target_x) + abs(y - target_y)
    return distance

In [24]:
def get_neighbors(puzzle):
    neighbors = []
    zero_pos = puzzle.index(0)
    zero_x, zero_y = zero_pos % 4, zero_pos // 4
    
    for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
        x, y = zero_x + dx, zero_y + dy
        if 0 <= x < 4 and 0 <= y < 4:
            new_pos = y * 4 + x
            new_puzzle = deepcopy(puzzle)
            new_puzzle[zero_pos], new_puzzle[new_pos] = new_puzzle[new_pos], new_puzzle[zero_pos]
            neighbors.append((new_puzzle, new_puzzle[zero_pos]))
    
    return neighbors

In [25]:
def solve_puzzle(puzzle):
    if not is_solvable(puzzle):
        return []
    
    goal = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0]
    heap = []
    heapq.heappush(heap, (manhattan_distance(puzzle), 0, puzzle, []))
    visited = set()
    visited.add(tuple(puzzle))
    
    while heap:
        _, cost, current, path = heapq.heappop(heap)
        
        if current == goal:
            return path
        
        for neighbor, move in get_neighbors(current):
            if tuple(neighbor) not in visited:
                visited.add(tuple(neighbor))
                new_cost = cost + 1
                priority = new_cost + manhattan_distance(neighbor)
                heapq.heappush(heap, (priority, new_cost, neighbor, path + [move]))
    
    return []

In [27]:
puzzle = [1,  2,  3,  4,
    5,  6,  7,  8,
    9,  0, 10, 12,
   13, 14, 11, 15]
solution_path = solve_puzzle(puzzle)
if solution_path:
    print("Найден путь решения (номера перемещённых плиток):", solution_path)
else:
    print("Решение не существует для данной расстановки.")

Найден путь решения (номера перемещённых плиток): [10, 11, 15]


### Вывод